In [17]:
# Imports used in this notebook.
import matplotlib as mpl
import ee
import folium
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from ee import batch
from matplotlib import cm
from google.colab import drive
from pprint import pprint as pp
from uuid import uuid1 as uniqueID

# Must authenticate your EE account before use of the package.
ee.Authenticate()
#ee.Initialize()
ee.Initialize(project='ksu-skytruth-vpca-automation', opt_url='https://earthengine-highvolume.googleapis.com')



In [28]:
import geemap
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.


  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  # water_pixels = (image.select('QA60').eq(0).
  qa = image.select('QA60').eq(0).And(image.select('SCL').eq(4))

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  return image.updateMask(mask).divide(10000)

dateRange = ['2023-11-10', '2023-11-20']

# #LW Field No Till
# [-81.15201, 41.018108]
# [-81.150265, 41.018474]
# [-81.150169, 41.021064]
# [-81.152002, 41.021079]

upperLeft = [-81.1583124,  41.0254715 ]
lowerRight = [-81.1469973,  41.0135215 ]
roi = ee.Geometry.Polygon(
  [[-81.15201, 41.018108],
   [-81.150265, 41.018474],
   [-81.150169, 41.021064],
   [-81.152002, 41.021079],
   [-81.15201, 41.018108]]
)
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2023-11-10', '2023-11-20').filterBounds(roi)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 55))
    .map(mask_s2_clouds)
)


visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}


m = geemap.Map()
m.set_center(-81.15113220018283, 41.01969153364965, 17)
m.add_layer(dataset.mean(), visualization, 'RGB')
m



Map(center=[41.01969153364965, -81.15113220018283], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:

# Define the polygon geometry
roi = ee.Geometry.Polygon(
  [[-81.15201, 41.018108],
   [-81.150265, 41.018474],
   [-81.150169, 41.021064],
   [-81.152002, 41.021079],
   [-81.15201, 41.018108]]
)

# Get the centroid of the polygon
centroid = roi.centroid()

# Print the coordinates of the centroid
centroid_coords = centroid.getInfo()['coordinates']
print("Centroid coordinates:", centroid_coords)

Centroid coordinates: [-81.15113220018283, 41.01969153364965]


In [20]:
link = dataset.mean().getDownloadURL({
    'scale': 10,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': roi})
print(link)

https://earthengine-highvolume.googleapis.com/v1/projects/ksu-skytruth-vpca-automation/thumbnails/b07a3f45f15f152e3fae6f261d9a8c61-b1b7077bac17525dd777ada141787c2c:getPixels
